In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import json
import re

#rank990()
- 990위까지의 상품의 상품번호와 상품명을 가져와서 csv파일로 저장하는 함수

In [ ]:
def rank990():

    items_num = []
    items_name = []

    for page in tqdm(range(11)):
      page += 1
      url = 'https://www.musinsa.com/ranking/best?period=month_3&age=ALL&mainCategory=&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page='+str(page)+'&viewType=small&priceMin=&priceMax=#pol2398497'
      resp = requests.get(url)
      soup = BeautifulSoup(resp.content, 'lxml')
      
      numbers = soup.select('#goodsRankList > li')
      names = soup.select('#goodsRankList > li > div.li_inner > div.article_info > p.list_info > a')
      
      for number, name in zip(numbers, names):
        items_num.append(number['data-goods-no'])
        items_name.append(name['title'])
      
      
    
    items = pd.DataFrame({'상품번호':items_num, '상품명':items_name})
    items.to_csv('/content/drive/MyDrive/csv/rank990.csv')


In [ ]:
rank990()

  0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# reviews()
- csv로 저장된 990위의 상품명과 상품번호를 불러와서 그 값을 이용해서 각 상품의 후기 개수와 마지막 페이지를 가져와서 csv 파일로 저장하는 함수

In [ ]:
def reviews():

  review_info = {
    '상품번호' : [],
    '상품명' : [],
    '후기' : [],
    '페이지' : []
  }
  
  items = pd.read_csv('/content/drive/MyDrive/csv/musinsa_rank_data.csv')
  items_name = items['상품명']
  items_num = items['상품번호']


  for item_num, item_name in tqdm(zip(items_num, items_name)):
    url = 'https://goods.musinsa.com/api/goods/v2/review/total-count?goodsSimilarNo=&goodsNo='+str(item_num)+'&selectedSimilarNo=0'

    headers = {
        'referer': 'https://store.musinsa.com/',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36 Edg/100.0.1185.29'}
    resp = requests.get(url, headers = headers)
    resp.text
    dic_resp = json.loads(resp.text)
    reviews_num = dic_resp['data']['goodsCount']
    last_page = reviews_num % 10
    if last_page == 0:
      last_pages = (reviews_num // 10)
    else:
      last_pages = (reviews_num // 10) + 1

    review_info['상품번호'].append(item_num)
    review_info['상품명'].append(item_name)
    review_info['후기'].append(reviews_num)
    review_info['페이지'].append(last_pages)

  df_review_info = pd.DataFrame(review_info)
  df_review_info.to_csv('/content/drive/MyDrive/csv/reviews.csv')
  


In [ ]:
reviews()

0it [00:00, ?it/s]

# users()
- csv로 저장된 상품명, 상품번호, 마지막 페이지 정보를 불러와서 각 상품들의 전체 일반 후기에서 사용자의 성별, 키, 몸무게, 구매한 사이즈, 평점, 구매날짜를 가져와 csv 파일로 저장하는 함수

In [ ]:
def users():

  reviews_open = pd.read_csv('/content/drive/MyDrive/csv/reviews.csv')
  items_name = reviews_open['상품명']
  items_num = reviews_open['상품번호']
  lastpages = reviews_open['페이지']


  for item_num, last_page in tqdm(zip(items_num, lastpages)):

    users_info = {
      '상품번호' : [],
      '성별' : [],
      '키' : [],
      '몸무게' : [],
      '구매한 사이즈' : [],
      '평점' :[],
      '구매날짜' : []
    }

    for page in tqdm(range(1, last_page+1)):
      url = 'https://goods.musinsa.com/api/goods/v1/review/goods/list?sort=new&page='+str(page)+'&goodsNo='+str(item_num)
      headers = {
          'referer' : 'https://store.musinsa.com/',
          'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36 Edg/100.0.1185.29'
      }
      resp = requests.get(url, headers = headers)
      soup = BeautifulSoup(resp.content, 'lxml')
      
      users = soup.select('#reviewListFragment > div > div.review-profile > div > div.review-profile__information')
      levels = soup.select('#reviewListFragment > div > div.review-profile > div > div.review-profile__text')
      stars = soup.select('#reviewListFragment > div > div.review-list__rating-wrap > span > span > span')
      sizes = soup.select('#reviewListFragment > div > div.review-goods-information > div.review-goods-information__item > p > span')

      test_list1 = []
      test_list2 = []
      test_list3 = []


      for user in users:
        if bool(user.select('p')):
          users = user.select('p')
          test_list1.append(users[0].text.split(',')[0])
          test_list2.append(users[0].text.split(',')[1].strip())
          test_list3.append(users[0].text.split(',')[2].strip())
        else:
          test_list1.append('**')
          test_list2.append('***cm')
          test_list3.append('**kg')

      for test1, test2, test3, level, star, size in zip(test_list1, test_list2, test_list3, levels, stars, sizes):
        users_info['상품번호'].append(item_num)
        users_info['성별'].append(test1)
        users_info['키'].append(test2)
        users_info['몸무게'].append(test3)
        users_info['평점'].append(int(star['style'][-4:-1])/20)
        users_info['구매날짜'].append(level.text.split('\n')[2])
        users_info['구매한 사이즈'].append(size.text.split('\n')[1].lstrip())
        if level.text.split('\n')[2] == '2021.12.31':
          break

      if level.text.split('\n')[2] == '2021.12.31':
        break
    
    df_users_info = pd.DataFrame(users_info)
    df_users_info.to_csv('/content/drive/MyDrive/csv/상품별 정보'+str(item_num)+'_info.csv')
    

    

In [ ]:
users()

In [ ]:
# 각 상품별로 info를 csv로 저장해 놓는게 나을듯
# 정보가 계속 갱신되니까 데이터 한번 저장하고 그 데이터를 이용할 데이터로 확정하고
# 데이터 전처리 과정으로 빨리 넘어가야 할듯!!!